# Mobile Get Input Notebook

This notebook automates creation of terrain profiles for ITU-R P.1812-6 propagation prediction.

## Workflow
1. **Setup**: Configure transmitter location and parameters
2. **Generate Receivers**: Create uniformly distributed receiver points
3. **Extract Profiles**: Get terrain elevation and land cover data
4. **Export**: Save profiles to CSV for batch processing

## Key Outputs
- `data/input/profiles/paths_oneTx_manyRx_Xkm.csv` - Terrain profiles ready for P1812
- `data/intermediate/api_data/lcm10_*.tif` - Cached land cover data
- `data/intermediate/workflow/rx_rings_*.csv` - Receiver point patterns

## Configuration
Edit the 'Transmitter Configuration' section below to set your TX location.


In [160]:
import os
import math
import requests
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point
import elevation
import rasterio
from rasterio.io import MemoryFile

from dataclasses import dataclass
from typing import Iterable, Optional, Union, List



In [ ]:
# Set up data paths
from pathlib import Path

# Detect project root
notebook_dir = Path.cwd()
project_root = Path.cwd() if (Path.cwd() / 'data').exists() else Path.cwd().parent

# Create data directories
profiles_dir = project_root / 'data' / 'input' / 'profiles'
api_data_dir = project_root / 'data' / 'intermediate' / 'api_data'
workflow_dir = project_root / 'data' / 'intermediate' / 'workflow'

profiles_dir.mkdir(parents=True, exist_ok=True)
api_data_dir.mkdir(parents=True, exist_ok=True)
workflow_dir.mkdir(parents=True, exist_ok=True)

print(f"Project root: {project_root}")
print(f"Profiles dir: {profiles_dir}")
print(f"API data dir: {api_data_dir}")
print(f"Workflow dir: {workflow_dir}")


## Transmitter Configuration

Edit the transmitter parameters below:


In [ ]:
"""
This notebook automate creation of the input for the ITU-R P.1812, based on the following implementation: https://github.com/eeveetza/Py1812 

* Mobile Simulation Tool *
Giga - Barcelona Tech Centre 

"""

### Define Transmitter Class


In [146]:
@dataclass

class Transmitter:
    tx_id: str
    lon: float
    lat: float
    htg: float
    f: float
    pol: int
    p: float
    hrg: float


In [148]:
tx = Transmitter(
    tx_id = "TX_0001",
    lon = -13.40694,
    lat = 9.345,
    htg = 57,
    f = 0.9,
    pol = 1,
    p = 50,
    hrg = 10,
)
tx

Transmitter(tx_id='TX_0001', lon=-13.40694, lat=9.345, htg=57, f=0.9, pol=1, p=50, hrg=10)

## Helper Functions

Define functions for receiver generation and profile extraction.


In [ ]:
def generate_receivers_radial_multi(
    tx,
    distances_km: Iterable[Union[int, float]],
    azimuths_deg: Iterable[Union[int, float]],
    *,
    start_rx_id: int = 1,
    include_tx_point: bool = False,
    crs: str = "EPSG:4326",
) -> gpd.GeoDataFrame:
    """
    Generate receivers on multiple rings around transmitter, based on a given distance and azimuths.
    
    Azimuth convention:
      - 0° = North
      - 90° = East
      - 180° = South
      - 270° = West
      - degrees increase clockwise

    Returns GeoDataFrame in EPSG:4326 with:
      tx_id, rx_id, link_id, distance_km, azimuth_deg, geometry
      
    Optionally include the transmitter as a point.
    """

    tx_gdf = gpd.GeoDataFrame(
        {"tx_id": [tx.tx_id]},
        geometry=[Point(tx.lon, tx.lat)],
        crs=crs,
    )
    utm_crs = tx_gdf.estimate_utm_crs()
    tx_utm = tx_gdf.to_crs(utm_crs)
    tx_pt = tx_utm.geometry.iloc[0]

    rows = []
    rx_id = start_rx_id

    # Optional: include transmitter itself
    if include_tx_point:
        rows.append({
            "tx_id": tx.tx_id,
            "rx_id": 0,
            "link_id": f"{tx.tx_id}_TX",
            "distance_km": 0.0,
            "azimuth_deg": None,
            "geometry": Point(tx.lon, tx.lat),
        })

    for d_km in distances_km:
        radius_m = float(d_km) * 1000.0

        for az in azimuths_deg:
            theta = math.radians(float(az))

            dx = radius_m * math.sin(theta)
            dy = radius_m * math.cos(theta)

            rx_utm = Point(tx_pt.x + dx, tx_pt.y + dy)
            rx_ll = gpd.GeoSeries([rx_utm], crs=utm_crs).to_crs(crs).iloc[0]

            rows.append({
                "tx_id": tx.tx_id,
                "rx_id": rx_id,
                "link_id": f"{tx.tx_id}_RX_{rx_id:04d}",
                "distance_km": float(d_km),
                "azimuth_deg": float(az),
                "geometry": rx_ll,
            })

            rx_id += 1

    return gpd.GeoDataFrame(rows, geometry="geometry", crs=crs)


In [161]:
distances = np.arange(0.5, 10.5, 0.5)   # 0.5, 1.0, 1.5, ... 10.0 km
azimuths = range(0, 360, 10)   # every 10°

receivers = generate_receivers_radial_multi(
    tx,
    distances,
    azimuths,
    include_tx_point=True
)


receivers.head()
receivers.to_csv("rx_rings_10deg_0_5km.csv")


In [131]:
#antenna's metadata 
f = 0.9 #frequency in GHz, in range 0.03 ≤ f ≤ 6
p = 50 #Time percentage for which the calculated basic transmission loss is not exceeded, default 50, but 1 ≤ p ≤ 50
pol = 1 #Polarization of the signal: 1 - horizontal, 2 - vertical
htg = 57 #antenna height
hrg = 10 #receiver height
lat = 9.345 #Latitude of a transmitter
lon = -13.40694 #Longitude of a transmitter


In [132]:
buffer_m = 11000        # 10 km radius
chip_px = 734            # ~30 m/pixel over 22 km (20,000/400=50)

In [ ]:
#metadata of the path between Tx/ Rx
max_distance_km = 11 #radius of the path, 
sampling_resolution = 30 #m
n_points = int(max_distance_km * 1000 / sampling_resolution) #considering that every step on the path should +-30m
path_azimuth = 0 #just a starting point
azimuths = list(range(0, 360, 10))  # change step: 5, 10, 15, etc. to create a circle grid around transmitter


In [ ]:
LCM10_TO_CT = {
    100: 1,  # Water
    80: 2,   # Bare/sparse
    30: 2,   # Grassland
    40: 2,   # Cropland
    70: 2,   # Moss/lichen
    110: 2,  # Snow/ice
    254: 2,  # No data
    20: 3,   # Shrubland
    50: 3,   # Herbaceous wetland
    10: 4,   # Tree cover
    60: 4,   # Mangroves
    90: 4,   # Built-up
}

CT_TO_R = {
    1: 0,   # Water
    2: 0,   # Open/rural
    3: 10,  # Suburban
    4: 15,  # Urban / trees / forest
    5: 20   # Dense urban
}

#optimization for future: take a look of the classes 

In [ ]:
from config_sentinel_hub import (
    SH_CLIENT_ID, SH_CLIENT_SECRET,
    TOKEN_URL, PROCESS_URL, COLLECTION_ID,
    DEFAULT_YEAR, DEFAULT_BUFFER_M, DEFAULT_CHIP_PX
)

#take out the api and secret into different file with all different apis and secrets.

# Imported from config_sentinel_hub.py
# Imported from config_sentinel_hub.py

# Imported from config_sentinel_hub.py
# Imported from config_sentinel_hub.py

# CLMS Land Cover 10m Annual V1 (LCM Global 10m Yearly V1)
# Imported from config_sentinel_hub.py

def meters_to_deg(lat, meters):
    dlat = meters / 111_320.0
    dlon = meters / (111_320.0 * math.cos(math.radians(lat)))
    return dlat, dlon

def get_token(client_id: str, client_secret: str) -> str:
    r = requests.post(
        TOKEN_URL,
        data={
            "grant_type": "client_credentials",
            "client_id": client_id,
            "client_secret": client_secret,
        },
        timeout=60,
    )
    r.raise_for_status()
    return r.json()["access_token"]

def landcover_at_point(
    client_id: str,
    client_secret: str,
    lat: float,
    lon: float,
    year: int = 2020,
    buffer_m: float = 1000,
    chip_px: int = 32,
    save_path: str | None = None,   # NEW

):
    token = get_token(client_id, client_secret)

    dlat, dlon = meters_to_deg(lat, buffer_m)
    bbox = [lon - dlon, lat - dlat, lon + dlon, lat + dlat]  # [minLon, minLat, maxLon, maxLat]

    evalscript = """
    //VERSION=3
    function setup() {
      return {
        input: ["LCM10"],
        output: { bands: 1, sampleType: "UINT8" }
      };
    }
    function evaluatePixel(s) {
      return [s.LCM10];
    }
    """

    body = {
        "input": {
            "bounds": {
                "bbox": bbox,
                "properties": {"crs": "http://www.opengis.net/def/crs/EPSG/0/4326"},
            },
            "data": [{
                "type": f"byoc-{COLLECTION_ID}",
                "dataFilter": {
                    "timeRange": {
                        "from": f"{year}-01-01T00:00:00Z",
                        "to":   f"{year}-12-31T23:59:59Z",
                    }
                },
            }],
        },
        "output": {
            "width": chip_px,
            "height": chip_px,
            "responses": [{"identifier": "default", "format": {"type": "image/tiff"}}],
        },
        "evalscript": evalscript,
    }

    r = requests.post(
        PROCESS_URL,
        json=body,
        headers={
            "Authorization": f"Bearer {token}",
            "Content-Type": "application/json",
            "Accept": "image/tiff",
        },
        timeout=120,
    )
    r.raise_for_status()

    # save GeoTIFF to disk
    if save_path:
        with open(save_path, "wb") as f:
            f.write(r.content)

    # Read GeoTIFF from memory
    with MemoryFile(r.content) as memfile:
        with memfile.open() as ds:
            arr = ds.read(1)  # uint8 codes

    center_code = int(arr[arr.shape[0] // 2, arr.shape[1] // 2])
    return center_code, arr

def resolve_credentials():
    """
    Priority:
    1) Environment variables SH_CLIENT_ID / SH_CLIENT_SECRET
    2) Constants SH_CLIENT_ID / SH_CLIENT_SECRET (fallback)
    """
    env_id = os.environ.get("SH_CLIENT_ID", "").strip()
    env_secret = os.environ.get("SH_CLIENT_SECRET", "").strip()

    if env_id and env_secret:
        return env_id, env_secret

    const_id = (SH_CLIENT_ID or "").strip()
    const_secret = (SH_CLIENT_SECRET or "").strip()

    # If you left placeholders, fail clearly
    if not const_id or not const_secret or "REPLACE_ME" in const_id or "REPLACE_ME" in const_secret:
        raise RuntimeError(
            "Credentials not found. Set SH_CLIENT_ID and SH_CLIENT_SECRET either as env vars "
            "or replace SH_CLIENT_ID/SH_CLIENT_SECRET constants in the script."
        )
    return const_id, const_secret

LCM10_CLASSES = {
    10: "Tree cover",
    20: "Shrubland",
    30: "Grassland",
    40: "Cropland",
    50: "Herbaceous wetland",
    60: "Mangroves",
    70: "Moss and lichen",
    80: "Bare / sparse vegetation",
    90: "Built-up (Urban)",
    100: "Permanent water bodies",
    110: "Snow and ice",
    254: "Unclassifiable / No data"
}

# CLUTTER_VALUES = {
#     "Water/sea": 0,
#     "Open/rural": 0,
#     "Suburban": 10,
#     "Urban/trees/forest": 15,
#     "Dense urban": 20
# }

if __name__ == "__main__":
    lat = lat
    lon = lon

    client_id, client_secret = resolve_credentials()

    buffer_m = 11000        # 10 km radius
    chip_px = 734            # ~30 m/pixel over 22 km (20,000/400=50)

    out_tif = f"lcm10_{lat}_{lon}_2020_buf{buffer_m}m_{chip_px}px.tif"

    code, chip = landcover_at_point(
        client_id, client_secret,
        lat, lon,
        year=2020,
        buffer_m=buffer_m,
        chip_px=chip_px,
        save_path=out_tif
    )

    class_name = LCM10_CLASSES.get(code, "Unknown class")
    print(f"Saved GeoTIFF: {out_tif}")
    print(f"Center land cover at ({lat}, {lon}) for 2020: {code} ({class_name})")


Saved GeoTIFF: lcm10_9.345_-13.40694_2020_buf11000m_734px.tif
Center land cover at (9.345, -13.40694) for 2020: 10 (Tree cover)


## Main Workflow Functions

These functions fetch data from APIs and extract terrain profiles.


In [ ]:
def generate_points_from_transmitter(
    lon: float,
    lat: float,
    max_distance_km: float,
    n_points: int,
    path_azimuth: float,
):
    if n_points < 2:
        raise ValueError("n_points must be >= 2 (include transmitter + at least one more point).")

    # Prepare elevation data lookup (downloads/caches tiles as needed)
    elevation_data = elevation.get_data()

    # 1) Transmitter point in WGS84
    tx = gpd.GeoDataFrame(geometry=[Point(lon, lat)], crs="EPSG:4326")

    # 2) Project to a metric CRS (UTM) so distances are in meters
    utm_crs = tx.estimate_utm_crs()
    tx_utm = tx.to_crs(utm_crs)
    center = tx_utm.geometry.iloc[0]

    # 3) Compute step distance
    max_m = max_distance_km * 1000.0
    step_m = max_m / (n_points - 1)

    # 4) Direction vector from bearing (clockwise from North)
    theta = math.radians(path_azimuth)
    dx_unit = math.sin(theta)
    dy_unit = math.cos(theta)

    points_utm = []
    distances_km = []

    for i in range(n_points):
        d_m = i * step_m
        x = center.x + d_m * dx_unit
        y = center.y + d_m * dy_unit
        points_utm.append(Point(x, y))
        distances_km.append(d_m / 1000.0)

    gdf_utm = gpd.GeoDataFrame(
        {"id": range(n_points), "d": distances_km, "path_azimuth": path_azimuth},
        geometry=points_utm,
        crs=utm_crs,
    )

    # 5) Back to WGS84 for elevation sampling (elevation.py expects lat/lon)
    gdf = gdf_utm.to_crs("EPSG:4326")

    zone = []
    # Load zones (GeoJSON)
    gdf_zones = gpd.read_file("zones_map_BR.json")

    # Make sure CRS matches points CRS
    if gdf_zones.crs != gdf.crs:
        gdf_zones = gdf_zones.to_crs(gdf.crs)

    # Spatial join (may produce duplicates if polygons overlap)
    gdf_joined = gpd.sjoin(
        gdf,
        gdf_zones[["zone_type_id", "geometry"]],
        how="left",
        predicate="intersects"  # or "within"
    )

    # Keep exactly one row per original point (drop duplicates created by overlap)
    gdf_joined = gdf_joined[~gdf_joined.index.duplicated(keep="first")]

    # Now this will match lengths
    gdf["zone"] = gdf_joined["zone_type_id"].fillna(0).astype(int).to_numpy()

    ct_codes = []
    tif_path = f"lcm10_{lat}_{lon}_2020_buf{buffer_m}m_{chip_px}px.tif"
    with rasterio.open(tif_path) as ds:
        band = ds.read(1)  # uint8 codes
        nodata = ds.nodata

        for geom in gdf.geometry:
            # rasterio uses (x=lon, y=lat)
            row, col = ds.index(geom.x, geom.y)

            if 0 <= row < ds.height and 0 <= col < ds.width:
                val = int(band[row, col])
                if nodata is not None and val == nodata:
                    val = 254  # treat nodata as "Unclassifiable/No data"
            else:
                val = 254  # outside tile bounds

            ct_codes.append(val)

    gdf["ct"] = ct_codes  # land cover codes at each point, taken from lcm10
    gdf["Ct"] = gdf["ct"].map(lambda c: LCM10_TO_CT.get(c, 2)) #matching required 1 - Water/sea, 2 - Open/rural, 3 - Suburban, 4 - Urban/trees/forest, 5 - Dense urban
    gdf["R"] = gdf["Ct"].map(lambda ct: CT_TO_R.get(ct, 0)).astype(int)



    # 6) Add elevation column (meters)
    h = []
    for geom in gdf.geometry:
        # elevation.py expects (lat, lon)
        z = elevation_data.get_elevation(geom.y, geom.x)
        h.append(0 if z is None else float(z))

    gdf["h"] = h
    return gdf


In [137]:
gdf = generate_points_from_transmitter(
        lon, lat,
        max_distance_km,
        n_points,
        path_azimuth
    )

gdf

/Users/aleksandra/Desktop/ITU/Mobile Simulation Tool/.venv/lib/python3.14/site-packages/pyogrio/raw.py:200: RuntimeWarning: Several features with id = 1 have been found. Altering it to be unique. This warning will not be emitted anymore for this layer
  return ogr_read(


,id,d,path_azimuth,geometry,zone,ct,Ct,R,h
0,0,0.000000,0,POINT (-13.40694 9.345),4,10,4,15,13
1,1,0.030137,0,POINT (-13.40694 9.34527),4,10,4,15,13
2,2,0.060274,0,POINT (-13.40694 9.34554),4,10,4,15,13
3,3,0.090411,0,POINT (-13.40694 9.34582),4,10,4,15,13
4,4,0.120548,0,POINT (-13.40694 9.34609),4,10,4,15,8
...,...,...,...,...,...,...,...,...,...
361,361,10.879452,0,POINT (-13.40649 9.44337),3,50,3,10,4
362,362,10.909589,0,POINT (-13.40649 9.44364),3,50,3,10,4
363,363,10.939726,0,POINT (-13.40649 9.44391),3,254,2,0,4
364,364,10.969863,0,POINT (-13.40649 9.44418),3,254,2,0,3


In [138]:
# 1) Extract Tx (first point) and Rx (last point) directly from gdf geometry
tx_geom = gdf.geometry.iloc[0]
rx_geom = gdf.geometry.iloc[-1]

lam_t = float(tx_geom.x)  # Tx longitude
phi_t = float(tx_geom.y)  # Tx latitude
lam_r = float(rx_geom.x)  # Rx longitude
phi_r = float(rx_geom.y)  # Rx latitude

h_profile = (
    gdf["h"]
    .fillna(0)
    .round()
    .astype(int)
    .tolist()
)

zone_profile = (
    gdf["zone"]
    .fillna(0)
    .round()
    .astype(int)
    .tolist()
)

ct_profile = (
    gdf["Ct"]
    .fillna(0)
    .round()
    .astype(int)
    .tolist()
)

R_profile = (
    gdf["R"]
    .fillna(0)
    .round()
    .astype(int)
    .tolist()
)

d_profile = gdf["d"].round(3).tolist()


## Extract and Export Profiles

Extract terrain profiles and save to CSV.


In [139]:
rows = []

for az in azimuths:
    gdf = generate_points_from_transmitter(
        lon, lat,
        max_distance_km,
        n_points,
        path_azimuth=az
    )

    # Tx / Rx coords
    tx_geom = gdf.geometry.iloc[0]
    rx_geom = gdf.geometry.iloc[-1]

    lam_t = float(tx_geom.x)
    phi_t = float(tx_geom.y)
    lam_r = float(rx_geom.x)
    phi_r = float(rx_geom.y)

    # Profiles
    h_profile_clean = [0 if v is None else int(round(v)) for v in gdf["h"].tolist()]
    ct_profile_clean = [0 if v is None else int(round(v)) for v in gdf["Ct"].tolist()]
    zone_profile_clean = [0 if v is None else int(round(v)) for v in gdf["zone"].tolist()]
    R_profile_clean = [0 if v is None else int(round(v)) for v in gdf["R"].tolist()]

    d_profile_clean = [float(round(v, 3)) for v in gdf["d"].tolist()]


    # One row per path
    rows.append({
        "f": float(f),
        "p": int(p),
        "d": d_profile_clean,
        "h": h_profile_clean,
        "R": R_profile_clean,
        "Ct": ct_profile_clean,
        "zone": zone_profile_clean,
        "htg": int(round(htg)),
        "hrg": int(round(hrg)),
        "pol": int(pol),
        "phi_t": phi_t,
        "phi_r": phi_r,
        "lam_t": lam_t,
        "lam_r": lam_r,
    })

df_all = pd.DataFrame(rows)


/Users/aleksandra/Desktop/ITU/Mobile Simulation Tool/.venv/lib/python3.14/site-packages/pyogrio/raw.py:200: RuntimeWarning: Several features with id = 1 have been found. Altering it to be unique. This warning will not be emitted anymore for this layer
  return ogr_read(
/Users/aleksandra/Desktop/ITU/Mobile Simulation Tool/.venv/lib/python3.14/site-packages/pyogrio/raw.py:200: RuntimeWarning: Several features with id = 1 have been found. Altering it to be unique. This warning will not be emitted anymore for this layer
  return ogr_read(
/Users/aleksandra/Desktop/ITU/Mobile Simulation Tool/.venv/lib/python3.14/site-packages/pyogrio/raw.py:200: RuntimeWarning: Several features with id = 1 have been found. Altering it to be unique. This warning will not be emitted anymore for this layer
  return ogr_read(
/Users/aleksandra/Desktop/ITU/Mobile Simulation Tool/.venv/lib/python3.14/site-packages/pyogrio/raw.py:200: RuntimeWarning: Several features with id = 1 have been found. Altering it to be

In [140]:
output_path = f"paths_oneTx_manyRx_{max_distance_km}km.csv"
df_all.to_csv(output_path, sep=";", index=False, decimal=".")
print(f"Saved {len(df_all)} paths to {output_path}")


Saved 36 paths to paths_oneTx_manyRx_11km.csv
